# A Two-Hours Seminar about [InterCom](https://github.com/Tecnologias-multimedia/intercom)

## 1. What is InterCom
InterCom is a real-time intercommunicator (currently only audio). The sequence of frames (two stereo samples) is splitted into chunks and each one is transmitted in an [UDP](https://en.wikipedia.org/wiki/User_Datagram_Protocol) packet.

## 2. Buffering
To hide the network [jitter](https://en.wikipedia.org/wiki/Jitter).

![Buffering](https://tecnologias-multimedia.github.io/study_guide/buffering/graphics/timelines.svg)

### Minimal implementation without buffering

In [ ]:
!python ../minimal.py --show_stats

### Show latency and jitter

In [ ]:
!ping -c 10 localhost

### Increases latency and jitter
* Latency = 100ms
* Jitter = 100ms
* Correlation between RTTs = 0.25 (<1)
* Statistical distribution of the RTTs = normal

In [ ]:
# Remember to add "your_username_here ALL=(ALL) NOPASSWD: ALL" to the end of /etc/sudoers
!tc qdisc show dev lo
!sudo tc qdisc add dev lo root handle 1: netem delay 100ms 100ms 25% distribution normal
!tc qdisc show dev lo

In [ ]:
!ping -c 10 localhost

### Let's listen again to minimal

In [ ]:
!python ../minimal.py --show_stats

### Let's listen to the version of InterCom that uses a buffer of 300 ms

In [ ]:
!python ../buffer.py -b 300 --show_stats

## 3. Compression using [DEFLATE](https://en.wikipedia.org/wiki/DEFLATE)
This step reduces the bit-rate. The chunks are compressed using [LZSS](https://en.wikipedia.org/wiki/Lempel-Ziv-Storer-Szymanski) (that is based on [LZ77](https://github.com/vicente-gonzalez-ruiz/LZ77)) and [Huffman Coding](https://en.wikipedia.org/wiki/Huffman_coding) (see also [this](https://vicente-gonzalez-ruiz.github.io/Huffman_coding/)). This text compression algorithm is used because:
1. Works well when repeated strings are found at the input.
2. Is included with [The Standard Python Library](https://docs.python.org/3/library/) ([zlib](https://docs.python.org/3/library/zlib.html)).

### Current throughput

In [ ]:
!sudo tc qdisc del dev lo root
!tc qdisc show dev lo
!ping -c 1 -s 65527 localhost # Header length: 9 bytes

In [ ]:
!printf "Gbps = "
!echo 65527*8/0.13/2/1000/1000 | bc -l

### Limit the transmission bit-rate to 200 kbps (and redefine the jitter)

In [ ]:
!sudo tc qdisc add dev lo root handle 1: netem delay 100ms 100ms 25% distribution normal
!sudo tc qdisc add dev lo parent 1:1 handle 10: tbf rate 200kbit burst 1024kbit limit 1024kbit

In [ ]:
!printf "kbps = "
!echo 65527*8/326.631/2 | bc -l

### Let's try again

In [ ]:
!python ../buffer.py -b 300 --show_stats

### Optional: remove all rules

In [ ]:
!sudo tc qdisc del dev lo parent 1:1 handle 10:
!sudo tc qdisc del dev lo root
!tc qdisc show dev lo

## 4. Bit-rate control through [quantization](https://github.com/vicente-gonzalez-ruiz/quantization/blob/master/digital_quantization.ipynb)

Quantization removes the less relevant information (mainly [electronic noise](https://en.wikipedia.org/wiki/Noise_(electronics)) ...) to increase the [compression ratio](https://en.wikipedia.org/wiki/Compression_ratio). A dead-zone quantizer is applied to the transform coefficients (generated after the spatial and the temporal decorrelation).

Controlling the quantization step, it's possible to control the transmission bit-rate. The current implementation measures the number of lost chunks per second and use this information to increase or decrease the quantization step.

### Adapting to the current throughput

In [ ]:
!python ../br_control.py -b 300 --show_stats

## 5. Spatial (inter-channel) decorrelation
The samples of a (stero) frame tend to have similar amplitudes. We apply [Mid/Side coding](https://tecnologias-multimedia.github.io/study_guide/spatial_decorrelation/).

## 6. Temporal (intra-channel) decorrelation
The samples of a channel exhibit temporal redundancy. We use a [Discrete Wavelet Transform (DWT)](https://tecnologias-multimedia.github.io/study_guide/temporal_decorrelation/) to exploit it.

## 7. Work to do
1. Currently, quantization minimizes the error in L$_2$ (the Mean Square Error). Perceptual quantization can increase the compression ratio without increasing the perceived distortion.
2. Transmit video.